In [8]:
import os

notebook_path = os.getcwd()
print(notebook_path)

/Users/anjalitiwari/ChatBot Training Model


This notebook is pre trained model for creating chatbot with good accuracy for this dataset. However, Accuracy range will be different.

You can add dataset with .csv file. But, you need to transfer your data json file into csv file format using python script. I tried it but don't seem to have converted successfully.

## **Import and load the data file**

We import the necessary packages for our chatbot and initialize the variables we will use in our Python project.
The data file is in JSON format so we used the json package to parse the JSON file into Python.

In [9]:
import nltk
nltk.download('punkt')#Sentence tokenizer




[nltk_data] Downloading package punkt to
[nltk_data]     /Users/anjalitiwari/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
!pip install nltk


In [11]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import warnings
warnings.filterwarnings('ignore')

In [12]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

# **Preprocessing**

In [13]:
import json
import csv

# Step 1: Load the JSON file
with open('intents.json', 'r') as json_file:
    data = json.load(json_file)

# Step 2: Extract data (assuming the JSON has a list of dictionaries structure)
# For example, [{"intent": "greeting", "patterns": ["Hi", "Hello"], "responses": ["Hi there!"]}, ...]

# Flatten the JSON structure for CSV conversion
rows = []
for item in data['intents']:  # Adjust based on the actual JSON structure
    intent = item.get('intent')
    for pattern in item.get('patterns', []):
        rows.append({"intent": intent, "pattern": pattern})

# Step 3: Write to CSV
with open('intents.csv', 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=["intent", "pattern"])
    writer.writeheader()  # Write column headers
    writer.writerows(rows)  # Write data rows

print("JSON to CSV conversion complete!")


JSON to CSV conversion complete!


In [14]:
import os
if os.path.exists('intents.json'):
    print("File exists!")
else:
    print("File not found.")



File exists!


In [15]:
import json

words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read() # read json file
intents = json.loads(data_file) # load json file


When working with text data, we need to perform various preprocessing on the data before we make a machine learning or a deep learning model. Based on the requirements we need to apply various operations to preprocess the 
data.
- Tokenizing is the most basic and first thing you can do on text data. 
- Tokenizing is the process of breaking the whole text into small parts like words.
- Here we iterate through the patterns and tokenize the sentence using nltk.word_tokenize() function and append each word in the words list. We also create a list of classes for our tags.

In [16]:
import re

def custom_tokenize(text):
    return re.findall(r'\b\w+\b', text)

for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = custom_tokenize(pattern)  # Custom tokenizer
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


In [17]:
import nltk
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/anjalitiwari/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [18]:
nltk.download('wordnet') #lexical database for the English language

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/anjalitiwari/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [19]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/anjalitiwari/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

Now we will lemmatize each word and remove duplicate words from the list. 
- Lemmatizing is the process of converting a word into its lemma form and then creating a pickle file to store the Python objects which we will use while predicting.

In [20]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents\n", documents, "\n")
# classes = intents[tag]
print (len(classes), "classes\n", classes, "\n")
# words = all words, vocabulary
print (len(words), "unique lemmatized words\n", words, "\n")
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

405 documents
 [(['Hi'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['What', 's', 'up'], 'greeting'), (['how', 'are', 'ya'], 'greeting'), (['heyy'], 'greeting'), (['whatsup'], 'greeting'), ([], 'greeting'), (['cya'], 'goodbye'), (['see', 'you'], 'goodbye'), (['bye', 'bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['I', 'am', 'Leaving'], 'goodbye'), (['Bye'], 'goodbye'), (['Have', 'a', 'Good', 'day'], 'goodbye'), (['talk', 'to', 'you', 'later'], 'goodbye'), (['ttyl'], 'goodbye'), (['i', 'got', 'to', 'go'], 'goodbye'), (['gtg'], 'goodbye'), (['what', 'is', 'the', 'name', 'of', 'your', 'developers'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'your', 'creators'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'the', 'developers'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'the', 'creators'], 'creator'), (['who', 'created', 'you'], 'cr

# **Training Model**

Now, we will create the training data in which we will provide the input and the output. 
- Our input will be the pattern and output will be the class our input pattern belongs to. But the computer doesn’t understand text so we will convert text into numbers

In [21]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words
    pattern_words = doc[0]
    # convert pattern_words in lower case
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create bag of words array, if word match found in current pattern then put 1 otherwise 0.[row * colm(263)]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # in output array 0 value for each tag and 1 value for matched tag.[row * colm(8)]
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

# shuffle training
random.shuffle(training)

# separate bags and output_rows
train_x = [item[0] for item in training]
train_y = [item[1] for item in training]

# convert to numpy arrays
train_x = np.array(train_x)
train_y = np.array(train_y)

print("Training data created")


Training data created


In [23]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

# **Build the model** 

We have our training data ready, now we will build a deep neural network that has 3 layers. We use the Keras sequential API for this. After training the model for 200 epochs, we achieved 100% accuracy on our model. Let us save the model as ‘chatbot_model.h5'.

In [24]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
print("First layer:",model.layers[0].get_weights()[0])

First layer: [[ 0.11104879 -0.03902665  0.07507138 ...  0.11869249 -0.11529727
  -0.03756327]
 [-0.04611936 -0.10379242  0.08750054 ... -0.11520318 -0.04149025
   0.08699208]
 [-0.08883289  0.03947431  0.10670197 ...  0.04588057 -0.01645825
   0.10332042]
 ...
 [-0.07670239  0.09471299  0.05733988 ...  0.12382676  0.00423077
  -0.12040383]
 [-0.00086717  0.01839148  0.01543343 ...  0.12162137 -0.0662228
   0.0683496 ]
 [ 0.01628427  0.04819246  0.11128312 ... -0.0748954  -0.03817131
   0.10383217]]


In [25]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [26]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


Epoch 1/200


2024-11-17 18:37:33.335062: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


81/81 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.0258 - loss: 3.6312
Epoch 2/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.1047 - loss: 3.5031
Epoch 3/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.1439 - loss: 3.3307
Epoch 4/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.2342 - loss: 3.0646
Epoch 5/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.3184 - loss: 2.7093
Epoch 6/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.3738 - loss: 2.5346
Epoch 7/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4147 - loss: 2.2293
Epoch 8/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4746 - loss: 2.0721
Epoch 9/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.5943 - loss: 1.7614
Epoch 10/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5462 - loss: 1.7236
Epoch 11/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6422 - loss: 1.5259
Epoch 12/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy:

model created


# FOR PREDICTING RESPONSE


# <b>For loading saved model</b>

In [27]:
from keras.models import load_model

model = load_model('chatbot_model.h5')

### **Predict the response**

To predict the sentences and get a response from the user to let us create a new file ‘chatapp.py’.
- We will load the trained model and then use a graphical user interface that will predict the response from the bot. The model will only tell us the class it belongs to, so we will implement some functions which will identify the class and then retrieve us a random response from the list of responses.
- Again we import the necessary packages and load the ‘words.pkl’ and ‘classes.pkl’ pickle files which we have created when we trained our model.

In [28]:
intents = json.loads(open('intents.json').read())

words = pickle.load(open('words.pkl', 'rb'))

classes = pickle.load(open('classes.pkl', 'rb'))


**To predict the class, we will need to provide input in the same way as we did while training. So we will create some functions that will perform text preprocessing and then predict the class**

In [30]:
#Utility Methods

def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    
    sentence_words = nltk.word_tokenize(sentence)
    #print(sentence_words)
    # stem each word - create short form for word
    
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    #print(sentence_words)
    
    return sentence_words
#return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    
    sentence_words = clean_up_sentence(sentence)
    #print(sentence_words)
    
    # bag of words - matrix of N words, vocabulary matrix
    
    bag = [0]*len(words) 
    #print(bag)
    
    for s in sentence_words:  
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
                #print ("found in bag: %s" % w)
    #print(bag)
    return(np.array(bag))
    
def predict_class(sentence, model):
    # filter out predictions below a threshold
    
    p = bow(sentence, words,show_details=False)
    #print(p)
    
    res = model.predict(np.array([p]))[0]
    #print(res)
    
    ERROR_THRESHOLD = 0.25
    
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    #print(results)
    # sort by strength of probability
    
    results.sort(key=lambda x: x[1], reverse=True)
    #print(results)
    
    return_list = []
    
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    
    return return_list
    #print(return_list)






**After predicting the class, we will get a random response from the list of intents:**



In [31]:

def getResponse(ints, intents_json):
    
    tag = ints[0]['intent']
    #print(tag)
    
    list_of_intents = intents_json['intents']
    #print(list_of_intents)
    
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result
    
def chatbot_response(text):
    ints = predict_class(text, model)
    #print(ints)
    
    res = getResponse(ints, intents)
    #print(res)
    return res

**Enter you queries** 

In [ ]:
start = True

while start:

    query = input('Enter Message:')
    if query in ['quit','exit','bye']:
        start = False
        continue
    try:
        res = chatbot_response(query)
        print(res)
    except:
        print('You may need to rephrase your question.')